# Sotavento: Problema de Clasificación
En el siguiente cuaderno se analiza el rendimiento del modelo escogido para producciones altas y bajas de energía
Tras ello, se transforma el problema original de predicción de producción de la energía en un problema de clasificación.

In [ ]:
#Importación de librerías
import pandas as pd


In [ ]:
#IMportación de datos del modelo final de regresión. 
import pickle
with open ('modelo_final.pkl', 'rb') as file:
    modelo_final = pickle.load(file)

# 1. Medición de rendimiento del modelo de regresión.
Tal y como nos indica el enunciado, se considera como producción alta aquella mayor que el tercer cuantil, y baja en otro caso. 
Para ello, primero añadimos una columna a nuestro dataset en función del valor de la energía. Esta nos permitirá realizar las mediciones de error y actuará como etiqueta en nuestro problema de clasificación más adelante. 
 

In [ ]:
#Importación de dataset
wind_ava=pd.read_csv('wind_ava.csv.gz', compression = "gzip")

wind_ava = wind_ava.set_index('datetime')

for c in wind_ava.columns:
    if not c.endswith('.13') and c != 'datetime':
        wind_comp = wind_ava.drop(c, axis = 1)

In [ ]:
#Inserción de la nueva columna. 
labels= []
dataset= 0
for i in dataset:
    if():
        labels.append("High")
    else:
        labels.append("Low")